## Alternative Process Plan Scheduling

This notebook demonstrates how to model and solve the classical Resource-Constrained Project Scheduling Problem with Alternative Process Plan
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.

### Ideas how to model it 

[Slides from the video](https://icaps17.icaps-conference.org/tutorials/T3-Introduction-to-CP-Optimizer-for-Scheduling.pdf)

Slide 65 -- Structural constraints

**Span constraint - (AND node)**

span(x,[y1,...,yn])

if x is present it spands all rpesent intervals from {y1,...yn}
that is. at least one of yi supports the start (resp. end) of x

if x is absent, then all yi are absent to

**Alternative constraint - (OR node)**

alterantive(x, [y1,...,yn])
if x is present, then **exactly one** of the {y1,..,yn} is present and synchronized with x (sa,e start and end value)

if x is absent then all yi are absent too

alternative(x, [y1,...,yn],k) k:integer expression is x is present then exaclt k of the {y1,...,yn} are present and synchronized with x (same start and end value) if x is absent, then all yi are absent too

combine with presenceOf(task[i]) => presenceOf(task[j])

Sequence constraints

Sequencing constraints are unary constraints on a sequence variable p
- first(p,x) : if x is present it is the first one in the permutation
- last(p,x) : if x is present it is the last one in the permutation
- prev(p,x,y) : if both x and y are present, x appears right before y in the permutation
- before(p,x,y) : if both x and y are present, x appears somewhere before y in the permutation

### Problem Definition

Let the production consist of $N$ indivisible tasks performed on specified machines according to a process plan. Define the set of non-preemptive tasks $T=\{0,\dots,N+1\}$ to be scheduled on a set of dedicated machines (renewable unary resources) indexed by $R=\{1,\dots,M\}$. Each task $i\in T$ has a processing time $PT_i$ and a machine assignment. Tasks $0$ and $N+1$ are dummy with $PT_0=PT_{N+1}=0$, where task $0$ is a predecessor and task $N+1$ is a successor of all other tasks. Precedence relations together with alternative process plans are specified via a Petri net (a directed bipartite graph with transitions and places). General temporal constraints (time lags) are given by a matrix $L=(\ell_{ij})$ over $T^2$, where each $\ell_{ij}\in\mathbb{R}$ enforces the inequality $s_i+\ell_{ij}\le s_j$, with $s_i$ the start time of task $i$. The earliest start time $ES_i$ is the minimum time at which task $i$ can be scheduled with respect to the temporal constraints alone (ignoring resources). We assume $\ell_{ij}\ge PT_i$ whenever transition $T_i$ is a direct predecessor of $T_j$ in the Petri net; all other $\ell_{ij}$ values are arbitrary. If there is no temporal constraint from $i$ to $j$, set $\ell_{ij}=-\infty$. If task $i$ has a release time $r_i$ and a deadline $\tilde d_i$, encode them as $\ell_{0i}=r_i$ and $\ell_{i0}=-\tilde d_i$ (with $s_0=0$).
Sequence-dependent setup times $o_{ij}$ represent additional time needed between tasks scheduled consecutively on the same machine. They satisfy the triangle inequality $o_{ij}+o_{jk}\ge o_{ik}$ for all $(i,j,k)\in T^3$. The optimality criterion is to minimize the schedule length (makespan) $C_{\max}$.

This problem can be classified as $PS_M,1,1\mid alt,\ temp,\ o_{ij}\mid C_{\max}$ in the $\alpha\mid\beta\mid\gamma$ notation, where $temp$ denotes generalized temporal constraints and $o_{ij}$ denotes setup times. Under the resource specification $PS_M,1,1$, there are $M$ machines with unary capacity, and each task requires at most one unit of a single machine. Since there are no additional resources, the problem can also be viewed as a machine-scheduling problem.